# Session

In [ ]:
#| default_exp session

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import csv

import os
import re
from pathlib import Path
from typing import Tuple, List, Union, Optional, Callable, Type, Iterable, no_type_check, Sequence
from IPython import display
from singleton_decorator import singleton
from numbers import Real
import pandas as pd
import os

import logging
logger = logging.getLogger(__name__)

from graph_rewrite import draw

from spannerlib.utils import checkLogs,get_base_file_path,assert_df_equals,DefaultIEs,DefaultAGGs
from spannerlib.grammar import parse_spannerlog,reconstruct
from spannerlib.span import Span
from spannerlib.data_types import (
    _infer_relation_schema,
     Var,
    FreeVar,
    RelationDefinition,
    Relation,
    IEFunction,
    AGGFunction,
    IERelation,
    Rule,
    pretty,
)
from spannerlib.engine import Engine

from spannerlib.micro_passes import (
    convert_primitive_values_to_objects,
    CheckReservedRelationNames,
    dereference_vars,
    check_referenced_paths_exist,
    inline_aggregation,
    relations_to_dataclasses,
    verify_referenced_relations_and_functions,
    rules_to_dataclasses,
    check_rule_safety,
    consistent_free_var_types_in_rule,
    assignments_to_name_val_tuple,
    statement_type_and_value,
    execute_statement,
)


In [ ]:
#| export
#| hide

def load_stdlib():
    # make sure we import the modules that register the stdlib
    import spannerlib.ie_func.json_path
    import spannerlib.ie_func.basic 





## Session class

In [ ]:
#| export
class Session():
    def __init__(self,register_stdlib=True,display_max_rows=10):
        
        self.pass_stack = [
            convert_primitive_values_to_objects,
            CheckReservedRelationNames('spanner_'),
            check_referenced_paths_exist,
            dereference_vars,
            inline_aggregation,
            relations_to_dataclasses,
            verify_referenced_relations_and_functions,
            rules_to_dataclasses,
            check_rule_safety,
            consistent_free_var_types_in_rule,
            assignments_to_name_val_tuple,
        ]

        self.clear(register_stdlib=register_stdlib)
        self.display_max_rows = display_max_rows

    def clear(self,register_stdlib=True):
        self.engine = Engine()
        if not register_stdlib:
            return
        load_stdlib()
        for ie_def in DefaultIEs().as_list():
            self.register(*ie_def)
        for agg_def in DefaultAGGs().as_list():
            self.register_agg(*agg_def)
    
    def register(self,name,func,in_schema,out_schema):
        ie_func_obj = IEFunction(name=name,func=func,in_schema=in_schema,out_schema=out_schema)
        self.engine.set_ie_function(ie_func_obj)

    def register_agg(self,name,func,in_schema,out_schema):
        agg_func_obj = AGGFunction(name=name,func=func,in_schema=in_schema,out_schema=out_schema)
        self.engine.set_agg_function(agg_func_obj)


    def _display_result(self,result,statement_lark):
        if result is None:
            pass
        elif isinstance(result,pd.DataFrame):
            with pd.option_context('display.max_rows', self.display_max_rows):
                display.display(reconstruct(statement_lark))
                display.display(
                    result.sort_values(by=list(result.columns))
                    .reset_index(drop=True)
                    .map(repr))
        elif isinstance(result,bool):
            display.display(reconstruct(statement_lark))
            display.display(result)
        else:
            pass
    


    def parse_and_check_semantics(self,code):
        statements = parse_spannerlog(code,split_statements=True)
        asts = []
        for statement_nx,statement_lark in statements:
            ast = statement_nx
            for pass_ in self.pass_stack:
                try:
                    pass_(ast,self.engine)
                except Exception as e:
                    raise Exception(
                        f"During semantic checks for statement \n\"{reconstruct(statement_lark)}\"\n"
                        f"in pass {pass_} the following exception was raised:\n{e}\n"
                        ).with_traceback(e.__traceback__)
            yield ast,statement_lark

    def format_results(self,res):
        if not isinstance(res,pd.DataFrame):
            return res
        if res.shape == (1,0):
            return True
        elif res.shape == (0,0):
            return False
        else:
            return res.sort_values(by=list(res.columns)).reset_index(drop=True)

    def plan_query(self,code):
        statements = list(self.parse_and_check_semantics(code))
        if len(statements) > 1:
            raise ValueError(f"Only one statement is allowed in plan_query, got {len(statements)}")
        ast,_ = statements[0]
        statement_node = list(ast.nodes)[0]
        node_data = ast.nodes[statement_node]
        statement = node_data['type']
        value = node_data['val']
        if statement != 'query':
            raise ValueError(f"Expected a query statement, got {statement}")

        query_graph,root =  self.engine.plan_query(value)
        return query_graph,root

    def draw_query(self,code):
        query_graph,root = self.plan_query(code)
        draw(query_graph)

    def execute_plan(self,query_graph,root,return_intermediate=False):
        res,inter = self.engine.execute_plan(query_graph,root,return_intermediate=True)
        res = self.format_results(res)
        if return_intermediate:
            return res,inter
        return res
        

    def export(self,code,display_results=False,return_statements_meta=False):
        results = []
        statements = []
        for clean_ast,statement_lark in self.parse_and_check_semantics(code):
            try:
                result = execute_statement(clean_ast,self.engine)
                result = self.format_results(result)
            except Exception as e:
                raise Exception(
                    f"During execution of statement \n\"{reconstruct(statement_lark)}\n\""
                    f"the following exception was raised:\n{e}\n"
                    ).with_traceback(e.__traceback__)
            
            s_type,s_dataclass = statement_type_and_value(clean_ast)
            statements.append((s_type,s_dataclass,reconstruct(statement_lark)))
            results.append(result)
            if display_results:
                self._display_result(result,statement_lark)
        
        if len(results) == 0:
            ret_val =  None
        else:
            ret_val =  results[-1]

        if return_statements_meta:
            return ret_val,statements
        else:
            return ret_val

    def import_rel(self,name:str,data:Union[str,Path,pd.DataFrame],delim:str = None):
        """Imports a relation into the current session, either from a dataframe or from a csv file."""
        if isinstance(data, (Path,str)):
            csv_file_name = Path(data)
            if not csv_file_name.is_file():
                raise IOError("csv file does not exist")
            if os.stat(csv_file_name).st_size == 0:
                raise IOError("csv file is empty")
            data = pd.read_csv(csv_file_name, delimiter=delim)

        first_row = list(data.iloc[0,:])
        scheme = _infer_relation_schema(first_row)
        rel_def = RelationDefinition(name=name,scheme=scheme)
        self.engine.set_relation(rel_def)
        self.engine.add_facts(name,data)

    def import_var(self,name,value):
        self.engine.set_var(name,value)
        
    def print_rules(self):
        rules = list(self.engine.rules_to_ids.keys())
        for rule in rules:
            print(rule)
        return rules
    def remove_rule(self,rule:str):
        self.engine.del_rule(rule)

    def remove_head(self,head:str):
        self.engine.del_head(head)

    def remove_all_rules(self):
        rules = list(self.engine.rules_to_ids.keys())
        for rule in rules:
            self.remove_rule(rule)

    def remove_relation(self,relation:str):
        self.engine.del_relation(relation)

    def get_all_functions(self):
        return {
            'ie':self.engine.ie_functions.copy(),
            'agg':self.engine.agg_functions.copy()
        }

### Test scaffold

In [ ]:
#| export
def test_session(
    queries,
    expected_outputs=None,# list of expected dfs
    ie_funcs=None,# List of [name,func,in_scheme,out_scheme]
    agg_funcs=None,
    csvs=None,# List of [name,df]
    debug=False,
    display_results=True,
    ):

    sess=Session()

    # add data
    if csvs:
        for name,df in csvs:
            sess.import_rel(name,df)
    # add ies
    if ie_funcs:
        for name,func,in_scheme,out_scheme in ie_funcs:
            sess.register(name,func,in_scheme,out_scheme)
    
    if agg_funcs:
        for name,func,in_scheme,out_scheme in agg_funcs:
            sess.register_agg(name,func,in_scheme,out_scheme)

    if not isinstance(queries,list):
        queries = [queries]
    if expected_outputs is None:
        expected_outputs = [None]*len(queries)
        dont_assert = True
    else:
        dont_assert = False
    if not isinstance(expected_outputs,list):
        expected_outputs = [expected_outputs]
    for query,expected in zip(queries,expected_outputs):
        if debug and query == queries[-1]:
            print(query)
            q,root = sess.plan_query(query)
            draw(q)
            res,inter = sess.execute_plan(q,root,return_intermediate=True)
            return res,inter,sess
        else:
            res = sess.export(query,display_results=display_results)
        # used for debugging, we return the results of the first query without expected
        # if expected is None:
        #     return query
        if dont_assert:
            continue
        if isinstance(expected,pd.DataFrame):
            assert_df_equals(res,expected)
        else:
            assert res == expected, f"expected {expected}, got {res}"
    return sess
        

## Tests

In [ ]:
# basic export with metadata
sess = Session()

res,meta = sess.export("""
new A(int)
A(1)
?A(X)
""",return_statements_meta=True)
display.display(res)

assert meta == [
    ('relation_declaration',
        RelationDefinition(name='A', scheme=[int]),
        'new A(int)'),
 ('add_fact', Relation(name='A', terms=[1], agg=None), 'A(1)'),
 ('query', Relation(name='A', terms=[FreeVar(name='X')], agg=None), '?A(X)')]

,X
0,1


In [ ]:
# test rule removal
sess = Session()
_ = sess.export('''
    new parent(str, str)
    new grandparent(str, str)
    parent("Liam", "Noah")
    parent("Noah", "Oliver")
    parent("James", "Lucas")
    parent("Noah", "Benjamin")
    parent("Benjamin", "Mason")
    grandparent("Tom", "Avi")
    ancestor(X,Y) <- parent(X,Y)
    ancestor(X,Y) <- grandparent(X,Y)
    ancestor(X,Y) <- parent(X,Z), ancestor(Z,Y)
''')


rules = sess.print_rules()
assert rules == ['ancestor(X,Y) <- parent(X,Y)',
'ancestor(X,Y) <- grandparent(X,Y)',
'ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)',]

sess.remove_rule("ancestor(X,Y) <- parent(X,Y)")
print("="*50)
rules = sess.print_rules()
assert rules == ['ancestor(X,Y) <- grandparent(X,Y)',
'ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)',]

ancestor(X,Y) <- parent(X,Y)
ancestor(X,Y) <- grandparent(X,Y)
ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)
ancestor(X,Y) <- grandparent(X,Y)
ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)


In [ ]:
# test clearing the engine
commands = """
    new parent(str, str)
    new grandparent(str, str)
    parent("Liam", "Noah")
    grandparent("Tom", "Avi")
    ancestor(X,Y) <- parent(X,Y)
    ancestor(X,Y) <- grandparent(X,Y)
    ancestor(X,Y) <- parent(X,Z), ancestor(Z,Y)
    """
session = Session()
output = session.export(commands)
session.print_rules()
session.clear()
assert session.print_rules() == []

ancestor(X,Y) <- parent(X,Y)
ancestor(X,Y) <- grandparent(X,Y)
ancestor(X,Y) <- parent(X,Z),ancestor(Z,Y)


In [ ]:
# importing relations from csv
session = Session()
session.import_rel(name="enrolled",data="./sample_data/enrolled.csv", delim=",")
commands = """
enrolled("abigail", "chemistry")
?enrolled(X,Y)
"""
res = session.export(commands)
assert_df_equals(res,pd.DataFrame([
    ["abigail", "chemistry"],
    ["gale", "operating_systems"],
    ["howard", "chemistry"],
    ["howard", "physics"],
    ["jordan", "chemistry"]
],columns=["X","Y"]))

,X,Y
0,abigail,chemistry
1,gale,operating_systems
2,howard,chemistry
3,howard,physics
4,jordan,chemistry


In [ ]:
# importing relations from dataframe
session = Session()
lecturer_df = pd.DataFrame(([["walter","chemistry"], ["linus", "operating_systems"]]))
session.import_rel("lecturer",lecturer_df)
commands = """ 
?lecturer(X,Y)
"""
res = session.export(commands)
assert_df_equals(res,pd.DataFrame([
    ["walter","chemistry"],
    ["linus", "operating_systems"]
],columns=["X","Y"]))


,X,Y
0,linus,operating_systems
1,walter,chemistry


In [ ]:
test_session(
    queries = [
        """
        new Parent(str, str)
        Parent("Sam", "Noah")
        Parent("Noah", "Austin")
        Parent("Austin", "Stephen")

        GrandParent(G, C) <- Parent(G, M), Parent(M, C)
        """,
        """?GrandParent(X, "Austin")"""
    ],
    expected_outputs = [
        None,
        pd.DataFrame({'X':['Sam']})
    ],
    # debug=True

)



'?GrandParent(X,"Austin")'

,X
0,'Sam'


<__main__.Session>

In [ ]:
# constants in rule heads
test_session(
    queries = [
        """
        new Parent(str, str)
        Parent("Sam", "Noah")
        Parent("Noah", "Austin")
        Parent("Austin", "Stephen")

        GrandParent(G, C) <- Parent(G, M), Parent(M, C)
        # all grand parents are fun
        FunGrandParent(G,C,"yes")<- GrandParent(G,C)
        """,
        """?FunGrandParent(X, "Austin",AreFun)"""
    ],
    expected_outputs = [
        None,
        pd.DataFrame([
            ["Sam","yes"]
        ],columns=["X","AreFun"])
    ],
    # debug=True

)



'?FunGrandParent(X,"Austin",AreFun)'

,X,AreFun
0,'Sam','yes'


<__main__.Session>

In [ ]:
def length(string: str) -> Iterable[int]:
        yield (len(string),)

_ =test_session(
    ["""new string(str)
    string("a")
    string("d")
    string("a")
    string("ab")
    string("abc")
    string("abcd")

    string_length(Str, Len) <- string(Str), Length(Str) -> (Len)

    """,
    """
    ?string_length(Str, Len)
    """],
    [
        None,
        pd.DataFrame({'Str':['a','d','ab','abc','abcd'],'Len':[1,1,2,3,4]}),
    ],
    ie_funcs=[
        ['Length',length,[str],[int]]
    ],
    # debug=True
)

'?string_length(Str,Len)'

,Str,Len
0,'a',1
1,'ab',2
2,'abc',3
3,'abcd',4
4,'d',1


In [ ]:
def ID(string: str):
        yield f'{string}_id',

def ID2(string: str):
        yield f'{string}_id2_z',f'{string}_id2_w'

In [ ]:
# empty input to ie functions
test_session(
        """
        new A(str, str)
        new B(str, str)
        A("1", "2")
        B("1", "3_id")
        C(X, Y) <- A(X, Y)
        D(X, Y, X) <- C(X, Y)
        # nothing will feed into ID but we still need output in the same schema as the first D rule
        D(X, Y, Z) <- A(X, "1"), B(X, Y), ID(X) -> (Y), ID2(Y)->(Z,W)
        ?D(X, Y, Z)
    """,
    pd.DataFrame([['1','2','1']],columns=['X','Y','Z']),
    ie_funcs=[
        ['ID',ID,[str],[str]],
        ['ID2',ID2,[str],[str,str]]
    ] 
)

'?D(X,Y,Z)'

,X,Y,Z
0,'1','2','1'


<__main__.Session>

In [ ]:
#multiple ie functions
test_session(
        """
        new A(str, str)
        new B(str, str)
        A("1", "2")
        B("1", "1_id")
        C(X, Y) <- A(X, Y)
        D(X, Y, X) <- C(X, Y)
        # nothing will feed into ID but we still need output in the same schema as the first D rule
        D(X, Y, Z) <- A(X, "2"), B(X, Y), ID(X) -> (Y), ID2(Y)->(Z,W)
        ?D(X, Y, Z)
    """,
    pd.DataFrame([('1', '1_id', '1_id_id2_z'), ('1', '2', '1')],columns=['X','Y','Z']),
    ie_funcs=[
        ['ID',ID,[str],[str]],
        ['ID2',ID2,[str],[str,str]]
    ] 
)


'?D(X,Y,Z)'

,X,Y,Z
0,'1','1_id','1_id_id2_z'
1,'1','2','1'


<__main__.Session>

In [ ]:
# ie functions bounded by constants
def split(string: str):
    for part in string.split():
        yield (part,)

test_session(
    ["""
    new String(str)
    String("he")
    String("hehe")
    Text(T) <- Split("he ho hehe hoho")->(T)
    StringFromText(S) <- String(S), Split("he ho hehe hoho")->(S)
    ?StringFromText(S)
    """,
    """?Text(T)"""
    ],
    [
        pd.DataFrame({'S':['he','hehe']}),
        pd.DataFrame({'T':['he','ho','hehe','hoho']})
    ],   
    ie_funcs=[
        ['Split',split,[str],[str]]
    
    ]
)

'?StringFromText(S)'

,S
0,'he'
1,'hehe'


'?Text(T)'

,T
0,'he'
1,'hehe'
2,'ho'
3,'hoho'


<__main__.Session>

In [ ]:
# Boolean queries

test_session(
    ["""
    new Parent(str, str)
    Parent("Sam", "Noah")
    Parent("Noah", "Austin")
    Parent("Austin", "Stephen")
    GrandParent(G, C) <- Parent(G, M), Parent(M, C)
    ?GrandParent("Sam", "Austin")
    """,
    """?GrandParent("Bob", "Austin")"""
    ],
    [
        True,
        False
    ]
)


'?GrandParent("Sam","Austin")'

True

'?GrandParent("Bob","Austin")'

False

<__main__.Session>

## Curr bug

In [ ]:
# use of ie functions with variable output arity
sess = test_session(
    ["""
    input_string = "John Doe: 35 years old, Jane Smith: 28 years old"
    AgeOf(Name,Age) <- rgx("(\w+\s\w+):\s(\d+)",$input_string) -> (NameSpan,AgeSpan),as_str(NameSpan)->(Name),as_str(AgeSpan)->(Age)
    ""","""
    ?AgeOf(X,Y)
    """],
    [
        None,
        pd.DataFrame({'X':["John Doe","Jane Smith"],'Y':["35","28"]}),
    ],
)

[(0, 8), (10, 12)]
[(24, 34), (36, 38)]


'?AgeOf(X,Y)'

,X,Y
0,'Jane Smith','28'
1,'John Doe','35'


In [ ]:
# q,r = sess.plan_query("?AgeOf(X,Y)")
# draw(q)

In [ ]:
# sess.execute_plan(q,r,return_intermediate=True)

In [ ]:
def format_ie(f_string,*params):
    yield f_string.format(*params),

string_schema = lambda x: ([str]*x)

test_session(
"""
new AgeOf(str, str)
AgeOf("John Doe", "35")
AgeOf("Jane Smith", "28")
age_description(Desc) <- AgeOf(Name, Age), format("{} is {} years old",Name,Age) -> (Desc)
?age_description(D)
""",
pd.DataFrame({'D':['Jane Smith is 28 years old','John Doe is 35 years old']}),
ie_funcs=[['format',format_ie,string_schema,[str]]]
)

'?age_description(D)'

,D
0,'Jane Smith is 28 years old'
1,'John Doe is 35 years old'


<__main__.Session>

In [ ]:
# test agg
# def format_ie(f_string,*params):
#     yield f_string.format(*params),

# string_schema = lambda x: ([str]*x)

test_session(
"""
new AgeOfKids(str, int)
AgeOfKids("John Doe", 35)
AgeOfKids("John Doe", 30)
AgeOfKids("Jane Smith", 28)
total_age(X,sum(Y)) <- AgeOfKids(X,Y)
?total_age(X,T)
""",
pd.DataFrame({'X':['John Doe','Jane Smith'],'T':[65,28]}),
)

'?total_age(X,T)'

,X,T
0,'Jane Smith',28
1,'John Doe',65


<__main__.Session>

In [ ]:
sess = Session()
df1 = pd.DataFrame([['John Doe', 35],['Jane Smith', 28]],columns=['X','Y'])
df2 = pd.DataFrame([['John Doe', 30]],columns=['X','Y'])

sess.import_rel("AgeOfKids",df1)
display.display(sess.export("?AgeOfKids(X,Y)"))
sess.import_rel("AgeOfKids",df2)
display.display(sess.export("?AgeOfKids(X,Y)"))

,X,Y
0,Jane Smith,28
1,John Doe,35


,X,Y
0,Jane Smith,28
1,John Doe,30
2,John Doe,35


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()
     